# AdaBoosting Algorithm From Scratch

## Importing Libraries

In [11]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier

## The AdaBoost Classifier

In [6]:
class MyAdaBoostClassifier:
    def __init__(self, n_estimators):
        self.n_estimators = n_estimators
        self.alphas = []
        self.models = []
        self.n_classes = None
    
    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        weights = np.full(X.shape[0], 1 / X.shape[0])
        
        for _ in range(self.n_estimators):
            class_models = []
            class_alphas = []
            
            for class_label in range(self.n_classes):
                # Create a binary label vector for the current class
                binary_labels = np.where(y == class_label, 1, -1)
                
                # Train a weak classifier
                model = DecisionTreeClassifier(max_depth=1)
                model.fit(X, binary_labels, sample_weight=weights)
                predictions = model.predict(X)
                
                # Calculate weighted error
                weighted_error = np.sum(weights * (predictions != binary_labels))
                
                # Calculate alpha
                alpha = 0.5 * np.log((1 - weighted_error) / (weighted_error + 1e-10)) + np.log(self.n_classes-1)
                class_alphas.append(alpha)
                
                # Update weights
                weights = weights * np.exp(-alpha * binary_labels * predictions)
                weights /= np.sum(weights)
                
                class_models.append(model)
            
            self.alphas.append(class_alphas)
            self.models.append(class_models)
    
    def predict(self, X):
        # Initialize scores for each class
        class_scores = np.zeros((self.n_estimators, self.n_classes, X.shape[0]))
        
        for index, (class_alphas, class_models) in enumerate(zip(self.alphas, self.models)):
            for class_label in range(self.n_classes):
                class_scores[index][class_label][:] += class_alphas[class_label] * class_models[class_label].predict(X)
        
        # Make multiclass predictions based on the highest score
        predictions = np.argmax(np.sum(np.sign(class_scores), axis=0), axis=0)

        return predictions

## Evaluating Our Model

### Iris Dataset

In [7]:
iris = pd.read_csv("iris.csv")
iris = iris.drop('Id', axis=1)

X = iris.iloc[:, 0:4]
y = iris['Species']
labels = {item: index for index, item in enumerate(np.unique(y))}
y = y.map(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2, shuffle=True)

model = MyAdaBoostClassifier(n_estimators=5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9666666666666667


### Breast Cancer Wisconsin (Diagnostic) Dataset

### AnnThyroid Dataset

In [ ]:
from pmlb import fetch_data

# Returns a pandas DataFrame
X1, y1 = fetch_data('ann_thyroid', return_X_y=True, local_cache_dir='./')

conditions = [y1 == 1, y1 == 2, y1 == 3]
values = [0, 1, 2]

y1 = np.select(conditions, values, default=y1)

X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=10, test_size=0.2, shuffle=True)

model = MyGradientBoostClassifier(n_estimators=0, learning_rate=0.1, max_depth=3)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
print(*y_test)
print(*predictions)

## Using AdaBoosting from library

### Iris Dataset

In [8]:
iris = pd.read_csv("iris.csv")  
iris = iris.drop('Id', axis=1)

X = iris.iloc[:, 0:4]
y = iris['Species']
labels = {item: index for index, item in enumerate(np.unique(y))}
y = y.map(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2, shuffle=True)

clf = AdaBoostClassifier(n_estimators=5)
clf.fit(X_train, y_train)
print("Accuracy:", clf.score(X_test, y_test))

Accuracy: 1.0


### Breast Cancer Wisconsin (Diagnostic) Dataset

### AnnThyroid Dataset